In [1]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: establishments-description.ipynb
#
# BY: Dmitry Sedov 
#
# CREATED: Wed Aug 6 2020
#
# DESC: This code constructs a dataset with stadiums and counts (visits) of
#       nearby establishments.
#
# EXEC:
#      
################################################################################
################################################################################

In [2]:
################################ Libraries #####################################

import sqlalchemy as db
import pandas as pd

import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

################################################################################

/home/user/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
################################# Options ######################################

pd.options.display.max_rows = 999

################################################################################

In [4]:
################################ Constants #####################################

year = 2018
project_folder = '/home/user/projects/stadiums'
output_folder = '/home/user/projects/stadiums/data/processed/descriptive'

################################################################################

In [5]:
# Import nearby establishment counts
def nearby_establishments(sport):
    """Read in the nearby establishments data for a sport."""
    
    # PostgreSQL connection
    engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
    
    # Create the stadiums with geo table
    create_temp_stadiums_table_with_geo = f"""
    CREATE TEMPORARY TABLE IF NOT EXISTS stadiums_with_geo AS
    SELECT
        sname_place_id,
        ST_SetSRID(ST_POINT(longitude, latitude), 4326)::geography AS s_location
    FROM
        stadiums
    WHERE 
        sport = '{sport}'
        ;

    CREATE INDEX IF NOT EXISTS stadiums_with_geo_sg_idx
    ON stadiums_with_geo (sname_place_id);

    CREATE INDEX IF NOT EXISTS stadiums_with_geo_location_idx
    ON stadiums_with_geo
    USING GIST(s_location);
    """
    
    # Create the all-establishments with geo table
    create_temp_places_table_with_geo = f"""
    CREATE TEMPORARY TABLE IF NOT EXISTS places_with_geo AS
    SELECT
        sname_place_id,
        naics_code / 10000 AS naics_2digit,
        ST_SetSRID(ST_POINT(longitude, latitude), 4326)::geography AS p_location
    FROM
        establishments
    UNION
    SELECT
        sname_place_id,
        naics_code / 10000 AS naics_2digit,
        ST_SetSRID(ST_POINT(longitude, latitude), 4326)::geography AS p_location
    FROM
        restaurants
    ;
    CREATE INDEX IF NOT EXISTS places_with_geo_sg_idx
    ON places_with_geo (sname_place_id);

    CREATE INDEX IF NOT EXISTS places_with_geo_location_idx
    ON places_with_geo
    USING GIST(p_location);
    """
    
    with engine.connect() as connection:
        result = connection.execute(create_temp_stadiums_table_with_geo)
    with engine.connect() as connection:
        result = connection.execute(create_temp_places_table_with_geo)
    
    # Get a summary of all establishments near stadiums
    summarize_places_close_to_stadiums = f"""
    WITH joined_by_distance AS (
    SELECT
        p.sname_place_id AS place_id,
        p.naics_2digit AS naics_2digit,
        s.sname_place_id AS stadium_id,
        ST_Distance(s.s_location, p.p_location) AS distance
    FROM
        stadiums_with_geo AS s
    LEFT JOIN
        places_with_geo AS p
    ON
        ST_DWithin(s.s_location, p.p_location, 5000)
    AND
        s.sname_place_id != p.sname_place_id )
    SELECT
        stadium_id,
        naics_2digit,
        distance_bin,
        COUNT(DISTINCT place_id) AS establishment_count,
        SUM(raw_visit_counts) AS total_establishment_visits
    FROM
        (SELECT 
            j.stadium_id,
            j.place_id,
            j.naics_2digit,
            CASE
                WHEN j.distance <= 1000 THEN 1
                WHEN j.distance > 1000 AND j.distance <= 2000 THEN 2
                WHEN j.distance > 2000 AND j.distance <= 3000 THEN 3
                WHEN j.distance > 3000 AND j.distance <= 4000 THEN 4
                WHEN j.distance > 4000 AND j.distance <= 5000 THEN 5
                ELSE null
            END AS distance_bin,
            v.raw_visit_counts
        FROM
            joined_by_distance AS j
        LEFT JOIN
            visits AS v
        ON
            j.place_id = v.sname_place_id
        AND
            year = {year}
        ) t
    GROUP BY
        stadium_id, naics_2digit, distance_bin
    ;
    """
    
    # Get the establishment count (visits) by industry and distance bin
    results = pd.read_sql(summarize_places_close_to_stadiums, con = engine)
    engine.dispose()
    
    return results

In [8]:
########################## Baseball stadiums localities ########################

baseball_establishments = nearby_establishments('baseball')

################################################################################

In [10]:
######################## Am. football stadiums localities ######################

football_establishments = nearby_establishments('football')

################################################################################

In [6]:
########################## Baseball stadiums localities ########################

basketball_establishments = nearby_establishments('basketball')

################################################################################

In [12]:
########################## Baseball stadiums localities ########################

hockey_establishments = nearby_establishments('hockey')

################################################################################

In [9]:
baseball_establishments['sports'] = 'baseball'
baseball_establishments_path = os.path.join(output_folder, 
                                            f'baseball_establishments_{year}.csv')
baseball_establishments.to_csv(path_or_buf = baseball_establishments_path,
                               index = False)

In [11]:
football_establishments['sports'] = 'football'
football_establishments_path = os.path.join(output_folder, 
                                            f'football_establishments_{year}.csv')
football_establishments.to_csv(path_or_buf = football_establishments_path,
                               index = False)

In [7]:
basketball_establishments['sports'] = 'basketball'
basketball_establishments_path = os.path.join(output_folder,
                                              f'basketball_establishments_{year}.csv')
basketball_establishments.to_csv(path_or_buf = basketball_establishments_path,
                               index = False)

In [13]:
hockey_establishments['sports'] = 'hockey'
hockey_establishments_path = os.path.join(output_folder,
                                          f'hockey_establishments_{year}.csv')
hockey_establishments.to_csv(path_or_buf = hockey_establishments_path,
                             index = False)